In [37]:
import os
import numpy as np
import pandas as pd
import lasio
import skfda
from skfda.representation.basis import BSpline
from skfda.representation.grid import FDataGrid

import matplotlib.pyplot as plt
from scipy import integrate
import scipy.stats as stats

from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from clustergram import Clustergram

In [ ]:
path = "train/"
train  = pd.read_csv(path + "train_data.csv").iloc[:,:-1]
train.head()

In [ ]:
mapping = {item:i for i, item in enumerate(train.WELL_NAME.unique())}
train.WELL_NAME = train.WELL_NAME.apply(lambda x: "well_"+str(mapping[x]))
train.head()

In [ ]:
# переименовываем названия las файлов которые есть в train

for i in os.listdir('LAS'):
    a = os.path.splitext(i)[0]
    try:
        os.rename('LAS/' + i, 'LAS/' + f'well{mapping[a]}.las')
    except:
        print(a)

In [ ]:
# находим виды каротажей, которые присутствуют во всех ГИС

col_list = []

for las_file in os.listdir('LAS'):
    if 'well' in las_file:
        las = lasio.read("LAS/" + las_file)
        df = las.df()
        col_list.append(list(df.columns))

col_list = list(set(col_list[0]).intersection(*col_list[1:]))
col_list

In [42]:
def find_num_basis(y_vals, patience_corr=3, threshold_corr=0.01, patience_area_delta=3, threshold_area_delta=0.1):
    
    """Функция для нахождения оптимального количества базисов для BSpline
    
    Параметры
    ----------
    y: array_like
       1D массив
    patience_corr: количество итераций без улучшений, после которых вычисление корреляции Пирсона будет остановлено. По умолчанию 3.
    threshold_corr: порог, при котором процентная разница между значениями корреляции Пирсона  будет считаться незначительной. По умолчанию 0.01 (1%).
    
    patience_area_delta: количество итераций без улучшений, после которых вычисление дельты будет остановлено. По умолчанию 3.
    threshold_area_delta: порог, при котором процентная разница между дельтами будет считаться незначительной. По умолчанию 0.1 (10%).
    
    Возвращает
    -------
    num_basis: оптимальное количество базисов
    """
    
    y = (y_vals - np.min(y_vals)) / (np.max(y_vals) - np.min(y_vals))
    
    grid_points = list(np.linspace(0, 1, len(y)))
    
    fd_orig = skfda.FDataGrid(data_matrix=y_vals, grid_points=grid_points)
    
    area_true = abs(integrate.simps(y=y, x=grid_points))
    
    func_data = []
    
    n_basis = 4
    const = 1

    corr_list = []
    prev_corr= None
    delta_corr_num = 0
    
    area_perc_list = []
    delta_area_num = 0
    
    basis_list = []
    
    while True:
        basis = BSpline(n_basis=n_basis)
        fd_basis_orig = fd_orig.to_basis(basis)
        
        data_prd_orig = fd_basis_orig.to_grid(grid_points).data_matrix.reshape(-1)
        current_corr = stats.pearsonr(data_prd_orig, y_vals)[0]
        
        data_prd_scal = (data_prd_orig - np.min(data_prd_orig)) / (np.max(data_prd_orig) - np.min(data_prd_orig))
        area_pred = abs(integrate.simps(y=data_prd_scal, x=grid_points))
        current_area_perc = 1 - (min(area_true, area_pred) / max(area_true, area_pred))
        
        if prev_corr is None:
            corr_list.append(current_corr)
            prev_corr = current_corr
            
            area_perc_list.append(current_area_perc)
            
            basis_list.append(n_basis)
            n_basis += const
            
        else:
            perc_corr_diff = 1 - (min(current_corr, prev_corr) / max(current_corr, prev_corr))
            
            if perc_corr_diff <= threshold_corr:
                delta_corr_num += 1
            else:
                delta_corr_num = 0
            
            if current_area_perc <= threshold_area_delta:
                delta_area_num += 1
            else:
                delta_area_num = 0
            
            corr_list.append(current_corr)
            prev_corr = current_corr
            
            area_perc_list.append(current_area_perc)
        
            basis_list.append(n_basis)
            n_basis += const
        
        if delta_area_num >= patience_area_delta and delta_corr_num >= patience_corr:
            break
    
    # # for mean
    ind = min(delta_corr_num, delta_area_num) + (abs(delta_corr_num - delta_area_num)//2)
    
    correlation_m = corr_list[-ind]
    delta_m = area_perc_list[-ind]
    n_basis_m = basis_list[-ind]
    
    basis = BSpline(n_basis=n_basis_m)
    fd_basis = fd_orig.to_basis(basis)
    
    # for delta
    delta_d = area_perc_list[-delta_area_num]
    correlation_d = corr_list[-delta_area_num]
    n_basis_d = basis_list[-delta_area_num]
    
    basis = BSpline(n_basis=n_basis_d)
    fd_basis = fd_orig.to_basis(basis)
    
    # # for corr
    delta_c = area_perc_list[-delta_corr_num]
    correlation_c = corr_list[-delta_corr_num]
    n_basis_c = basis_list[-delta_corr_num]
    
    basis = BSpline(n_basis=n_basis_c)
    fd_basis = fd_orig.to_basis(basis)
    
    num_basis = max([n_basis_d, n_basis_c, n_basis_m])
    
    return num_basis

### Поиск оптимального количества базисов для всех видов каротажей

1. Для каждой кривой находится оптимальное количество базисов
2. Для всех кривых применяется аппроксимация BSpline с максимальным количеством базисов найденным среди кривых
3. Получаем коэффициенты, характеризующие кривые

In [ ]:
lst_files = os.listdir('LAS')

coeffs = []

for col_name in col_list:
    
    las_data = []
    
    for las_file in lst_files:
        
        if 'well' in las_file:
            las = lasio.read("LAS/" + las_file)
            df = las.df()
            sc = MinMaxScaler(feature_range=(-1,1))
            df = sc.fit_transform(df[col_name].values.reshape(-1, 1)).reshape(-1)
            las_data.append(df)

    mx_len = len(max(las_data, key=len))
    grid_points_intpl = np.linspace(0, 1, mx_len)

    n_basis_lst = []
    intrp_curves = []
    
    for curve in las_data:
        grid_points = np.linspace(0, 1, len(curve))
        a = FDataGrid(data_matrix=curve, grid_points=grid_points).to_grid(grid_points_intpl)
        n_bas  = find_num_basis(curve)
        # n_bas  = find_num_basis(a.data_matrix.reshape(-1))
        n_basis_lst.append(n_bas)
        intrp_curves.append(a)
    
    n_bas = max(n_basis_lst)
    
    data = 0

    for lst in intrp_curves:
        if data == 0:
            data = lst
        else:
            data = data.concatenate(lst)
            
    print(col_name)
    print('number of basis', n_bas)
    data.plot()
    plt.show()
    
    basis = BSpline(n_basis=int(n_bas))
    data_basis = data.to_basis(basis)
    data_basis.plot()
    plt.show()

    coeffs.append(data_basis.coefficients)

In [44]:
# объединяем коэффициенты по всем видам каротажей в один вектор признаков

dat = []

for lst_1, lst_2, lst_3 in zip(coeffs[0], coeffs[1], coeffs[2]):
    dat.append(np.hstack((lst_1, lst_2, lst_3)))

coeffs = dat

In [ ]:
from sklearn.decomposition import PCA
import plotly.express as px


# снижаем размерность с потерей информации не выше 10%
for num_comptnts in range(2, len(coeffs[0])):
    
    dd = pd.DataFrame(coeffs)

    pca = PCA(n_components=num_comptnts)
    pca_data = pca.fit_transform(dd)

    total_var = pca.explained_variance_ratio_.sum() * 100
    
    if total_var >= 90:
        print(f'number of components: {num_comptnts}, with total_var={total_var}')
        break

pca_data

### Кластеризация

In [ ]:
cgram = Clustergram(range(1, 7))
cgram.fit(np.array(coeffs))
cgram.plot()

In [ ]:
cgram = Clustergram(range(1, 7), method='hierarchical', linkage='ward')
cgram.fit(np.array(coeffs))
cgram.plot()

In [ ]:
distortions = []
silhouette = []

K = range(1, 10)
for k in K:
    kmeanModel = KMeans(n_clusters=k)
    kmeanModel.fit(coeffs)
    distortions.append(kmeanModel.inertia_)
    if k > 1:
        silhouette.append(silhouette_score(coeffs, kmeanModel.labels_))

plt.figure(figsize=(10,4))
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('Elbow Method')
plt.show()

plt.figure(figsize=(10,4))
plt.plot(K[1:], silhouette, 'bx-')
plt.xlabel('k')
plt.ylabel('Silhouette score')
plt.title('Silhouette')
plt.show()

In [49]:
# используем переменную coeffs для кластеризации

result = KMeans(2).fit_predict(coeffs)

### Визуализация результатов кластеризации с помощью PCA

In [ ]:
dd = pd.DataFrame(coeffs)

pca = PCA(n_components=2)

pca_data = pca.fit_transform(dd)

dd = pd.DataFrame(pca_data[:, 0])

dd[1] = pca_data[:, 1]

dd[2] = result

dd[2] = dd[2].astype("category")

fig = px.scatter(dd, x=0, y=1,color=2)
fig.show()

In [ ]:
dd = pd.DataFrame(coeffs)

pca = PCA(n_components=3)
pca_data = pca.fit_transform(dd)
total_var = pca.explained_variance_ratio_.sum() * 100
dd = pd.DataFrame(pca_data[:, 0])

dd[1] = pca_data[:, 1]

dd[2] = pca_data[:, 2]

dd[3] = result

dd[3] = dd[3].astype("category")

fig = px.scatter_3d(dd, x=0, y=1, z=2, color=3,
                    title=f'Total Explained Variance: {total_var:.2f}%'
                    )
fig.show()